In [1]:
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=10,
)

examples = [
    {
        "question": "I want to represent the movie 'Top Gun' with three emojis.",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "I want to represent the movie 'The Godfather' with three emojis.",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "I want to represent the movie 'The Martian' with three emojis.",
        "answer": "🧑‍🚀🚀🥔",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie critic. You should answer 3 emoji that represents the movie well.",
        ),

        MessagesPlaceholder(variable_name="history"),
        chat_prompt,
        (
            "human",
            "{question}",
        ),
    ]
)

def save(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )


save("I want to represent the movie 'Top Gun' with three emojis.", "🛩️👨‍✈️🔥")
save("I want to represent the movie 'The Godfather' with three emojis.", "👨‍👨‍👦🔫🍝")
save("I want to represent the movie 'The Martian' with three emojis.", "🧑‍🚀🚀🥔")


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_movie(movie):
    question = f"I want to represent the movie '{movie}' with three emojis."
    result = chain.invoke({"question": question})
    save(question, result.content)


def invoke_question(question):
    result = chain.invoke({"question": question})
    save(question, result.content)


In [2]:
invoke_movie("Interstellar")

🌌🚀⏳

In [3]:
invoke_movie("Spider-man: No Way Home")

🕷️🏙️🌌

In [5]:
invoke_movie("Alien: Romulus")

👽🚀🔫

In [7]:
invoke_question("Can you tell me the movies I asked before?")

Certainly! Here are the movies you asked about along with their emoji representations:

1. **Top Gun**: 🛩️👨‍✈️🔥
2. **The Godfather**: 👨‍👨‍👦🔫🍝
3. **The Martian**: 🧑‍🚀🚀🥔
4. **Interstellar**: 🌌🚀⏳
5. **Spider-Man: No Way Home**: 🕷️🏙️🌌
6. **Alien: Romulus**: 👽🚀🔫

Let me know if you need anything else!